In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import random
import string
import hashlib
import json

#Loading original data
columns_name = ['id', 'date', 'latitude', 'longitude' ]

df = pd.read_csv('../Data_Anom/real_data_set.csv', names = columns_name, sep = '\t')
df['date'] = df['date'].str.slice(stop = 16)
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d %H:%M")
df['week'] = df['date'].dt.isocalendar().week

In [2]:
anon = pd.read_csv('../Data_Anom/Lbitrik_534', names = columns_name, sep = '\t')
anon = anon.loc[anon.id != 'DEL']
anon['date'] = anon['date'].str.slice(stop = 16)
anon['date'] = pd.to_datetime(anon['date'], format = "%Y-%m-%d %H:%M")
anon['week'] = anon['date'].dt.isocalendar().week
anon.latitude = anon.latitude.astype(float)
anon.longitude = anon.longitude.astype(float)

In [3]:
print(anon)

                       id                date  latitude  longitude  week
0         72ada0500aa0c37 2015-03-17 23:00:00  4.866057  45.755805    12
1         3d48eb529410205 2015-03-22 11:00:00  4.852817  45.738120    12
2         9c682b41b971b29 2015-04-12 13:00:00  4.866397  45.764273    15
3         f80a1429d85f0db 2015-04-15 11:00:00  4.589533  45.819687    16
4         f3a6abf083aca13 2015-03-16 16:00:00  4.880665  45.783387    12
...                   ...                 ...       ...        ...   ...
34551841  70c55d75d8edd7b 2015-04-27 00:00:00  4.865930  45.755895    18
34551843  d4a71d3ff633668 2015-03-10 11:00:00  4.950113  45.691112    11
34551844  f40b75581a9f24f 2015-05-02 11:00:00  4.756413  45.995440    18
34551846  ab24cc0a1b936cb 2015-03-08 16:00:00  4.756338  45.995467    10
34551847  fab3705409406af 2015-03-19 16:00:00  4.756403  45.995440    12

[29971332 rows x 5 columns]


In [4]:
print(df)

           id                date  latitude  longitude  week
0           1 2015-03-04 00:35:00  4.870147  45.772140    10
1           1 2015-03-04 00:35:00  4.870218  45.772095    10
2           1 2015-03-04 00:35:00  4.870210  45.772072    10
3           1 2015-03-04 00:35:00  4.870210  45.772072    10
4           1 2015-03-04 00:35:00  4.870210  45.772072    10
...       ...                 ...       ...        ...   ...
34551844  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551845  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551846  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551847  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551848  110 2015-03-12 19:29:00  2.343127  48.891772    11

[34551849 rows x 5 columns]


In [5]:
#List of id origin, id anonym, group of id-week

id_week_ori = df.groupby('id')['week'].unique()
id_week_anon = anon.groupby('id')['week'].unique()

list_id_ori = df["id"].unique()
list_id_anon = anon["id"].unique()

print(np.sort(list_id_ori), len(list_id_ori))
print(np.sort(list_id_anon), len(list_id_anon))

[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  18  21  23  24
  25  26  27  28  29  30  31  32  34  35  36  37  38  39  41  42  43  44
  48  49  50  51  52  53  54  55  58  59  60  62  63  65  66  67  68  69
  70  71  72  73  75  77  78  81  83  84  87  89  98 107 110] 69
['0020b80195eff75' '0120c190986ffca' '0125e475881a52d' '02529dae41df7c3'
 '03f32f203caf611' '03fdc34d2dabf41' '058787dde0a0c81' '05b2822721e49da'
 '05d758838d8ec22' '063d02325be1a72' '0640bb9ee51d621' '06dce8b3da72dc1'
 '0724705ec0fabfd' '073f0797548aadc' '07f516ec7a79015' '096bcefa7481583'
 '09ffcd01c5e3c89' '0ad7eccc801d9f7' '0b4ff693b4bb6ae' '0b9ed3b00edbcfd'
 '0bb5c4b2b2b6bcb' '0c2552357f1069e' '0d3897225d4d328' '0d4003618845673'
 '0e5496d9d67c10e' '0eb75922a6f1a06' '0ef74def2dc1316' '0f36e6c97b4583b'
 '0fe3635e9f7cdc2' '105b7f038434f0f' '1099e2b09bdc723' '109f05835bf7de5'
 '11e06154553f9d0' '11e21d0c7854697' '1215c1174b08367' '1255ee3f68411fa'
 '13111d0a78734b4' '1339fe940dd24c9' '138ad9edc9708a9' '13f

In [6]:
#Group id--week--moyen_latitude--moyen_longitude--mean_date--count_line--mean_hour

#Original Table
df['hour'] = df['date'].dt.hour
mean_hour = df.groupby(['id','week'])['hour'].mean()

mean_lat_ori = df.groupby(['id','week'])['latitude'].mean()
mean_long_ori = df.groupby(['id','week'])['longitude'].mean()

mean_date_ori = df.groupby(['id','week'])['date'].mean()
count_ligne_ori = df.groupby(['id','week'])['date'].count()
mean_hour_ori = df.groupby(['id','week'])['hour'].mean()

ori_mean = df[['id','week']].drop_duplicates().reset_index(drop=True)
ori_mean['mean_latitude'] = ori_mean.apply(lambda x: mean_lat_ori[x.id][x.week], axis=1)
ori_mean['mean_longitude'] = ori_mean.apply(lambda x: mean_long_ori[x.id][x.week], axis=1)
ori_mean['mean_date'] = ori_mean.apply(lambda x: mean_date_ori[x.id][x.week], axis=1)
ori_mean['count_line'] = ori_mean.apply(lambda x: count_ligne_ori[x.id][x.week], axis=1)
ori_mean['mean_hour'] = ori_mean.apply(lambda x: mean_hour[x.id][x.week], axis=1)

#Anonymisation Table
anon['hour'] = anon['date'].dt.hour
mean_lat_anon = anon.groupby(['id','week'])['latitude'].mean()
mean_long_anon = anon.groupby(['id','week'])['longitude'].mean()
mean_date_anon = anon.groupby(['id','week'])['date'].mean()
count_ligne_anon = anon.groupby(['id','week'])['date'].count()
mean_hour_anon = anon.groupby(['id','week'])['hour'].mean()

anon_mean = anon[['id','week']].drop_duplicates().reset_index(drop=True)
anon_mean['mean_latitude'] = anon_mean.apply(lambda x: mean_lat_anon[x.id][x.week] , axis=1)
anon_mean['mean_longitude'] = anon_mean.apply(lambda x: mean_long_anon[x.id][x.week], axis=1)
anon_mean['mean_date'] = anon_mean.apply(lambda x: mean_date_anon[x.id][x.week], axis=1)
anon_mean['count_line'] = anon_mean.apply(lambda x: count_ligne_anon[x.id][x.week], axis=1)
anon_mean['mean_hour'] = anon_mean.apply(lambda x: mean_hour_anon[x.id][x.week], axis=1)

In [22]:
ori_mean['new_id'] = ori_mean['id']
scale = anon.shape[0] / df.shape[0]
for i in list_id_ori:
    for w in id_week_ori[i]:
        list_distance = list() 
        mean_lat = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_latitude'].to_list()[0]
        mean_long = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_longitude'].to_list()[0]
        mean_date = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_date'].to_list()[0]
        number_line = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'count_line'].to_list()[0]
        mean_hour = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_hour'].to_list()[0]
        
        group_week = anon_mean.loc[anon_mean.week ==w,['id','week','mean_latitude','mean_longitude','count_line','mean_date','mean_hour']].reset_index(drop=True)
        for j in range(len(group_week)):
            distance = np.power((group_week.mean_latitude[j] - mean_lat)**2 + (group_week.mean_longitude[j] - mean_long)**2, 1/2)
            
            delta_ratio = np.power(abs((group_week.count_line[j]/anon.shape[0] - number_line/df.shape[0])),4)
            delta_time = np.power(abs((group_week.mean_date[j] - mean_date).total_seconds() / 3600),1)
            
            
            delta_hour = np.power(abs((group_week.mean_hour[j] - mean_hour)) ,1/2)
            
            list_distance.append(np.power(distance,5)*delta_ratio*delta_hour)
            
        index = list_distance.index(min(list_distance))
        l = id_week_anon[group_week.id[index]]# list of week have id anon_mean.id[index] 
        while ((w not in l)):
            list_distance[index] = 1000000000
            index = list_distance.index(min(list_distance).all())
            l = id_week_anon[group_week.id[index]]
        if (w in l):
            ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'] = group_week.id[index]
        
#print(ori_mean.head())
print(ori_mean.new_id.nunique())

result = {}
count2 = 0
for i in list_id_ori:
    result[str(i)] = {}
    for w in id_week_ori[i]:
        result[str(i)]['2015'+'-'+str(w)] = [str(ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'][count2])]
        count2 +=1
print(result)

413
{'1': {'2015-10': ['e5c3462c8e7d6a8'], '2015-11': ['790d1b68ae4f432'], '2015-12': ['1f9db186363af11'], '2015-13': ['58013aed3bc75d6'], '2015-14': ['8786269ee7b7138'], '2015-15': ['9e79e009bedbd5a'], '2015-16': ['99cfe5c7a984385'], '2015-17': ['d0dea4575394a5d'], '2015-18': ['8067b1d3814035b'], '2015-19': ['febff48056fee9e'], '2015-20': ['2e11995df216e2f']}, '2': {'2015-10': ['07f516ec7a79015'], '2015-11': ['6322086177a90fb'], '2015-12': ['3f31ad5a8268952'], '2015-13': ['a0d7819b1a80e4b'], '2015-14': ['442e910cf47398a'], '2015-15': ['0ef74def2dc1316'], '2015-16': ['2a6c5e810e401b8'], '2015-17': ['55d95c441459e43'], '2015-18': ['ff1cfd4378d97a7'], '2015-19': ['c94aef755183d8e'], '2015-20': ['1aec4066f5a9466']}, '107': {'2015-10': ['8b2af2a2aa3b736'], '2015-11': ['ce3e7d240a62458'], '2015-12': ['50c7189ceaf46da']}, '98': {'2015-10': ['073f0797548aadc'], '2015-11': ['0120c190986ffca']}, '21': {'2015-10': ['60cc3f39bcaf795'], '2015-11': ['b50add909968a96']}, '5': {'2015-10': ['8b832aeb4

In [23]:
jsonData = json.dumps(result, indent=4)
print(jsonData)

{
    "1": {
        "2015-10": [
            "e5c3462c8e7d6a8"
        ],
        "2015-11": [
            "790d1b68ae4f432"
        ],
        "2015-12": [
            "1f9db186363af11"
        ],
        "2015-13": [
            "58013aed3bc75d6"
        ],
        "2015-14": [
            "8786269ee7b7138"
        ],
        "2015-15": [
            "9e79e009bedbd5a"
        ],
        "2015-16": [
            "99cfe5c7a984385"
        ],
        "2015-17": [
            "d0dea4575394a5d"
        ],
        "2015-18": [
            "8067b1d3814035b"
        ],
        "2015-19": [
            "febff48056fee9e"
        ],
        "2015-20": [
            "2e11995df216e2f"
        ]
    },
    "2": {
        "2015-10": [
            "07f516ec7a79015"
        ],
        "2015-11": [
            "6322086177a90fb"
        ],
        "2015-12": [
            "3f31ad5a8268952"
        ],
        "2015-13": [
            "a0d7819b1a80e4b"
        ],
        "2015-14": [
            "442e91